<a href="https://colab.research.google.com/github/MinamiNaoya/Audio_to_Notion/blob/master/wav2vec2learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!sudo apt update
!sudo apt upgrade -y

In [2]:
%%capture
!pip install --upgrade pyarrow

!pip install transformers==4.11.3
!pip install torchaudio
!pip install librosa
!pip install jiwer




In [3]:
%%capture
!pip3 install datasets

## 参考文献
https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb#scrollTo=c8eh87Hoee5d

In [4]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_11_0", "ja", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0", "ja", split="test")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6505it [00:00, 54047.63it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4485it [00:00, 57046.07it/s]


...


Reading metadata...: 4604it [00:00, 68848.56it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1146it [00:00, 54082.91it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2464it [00:00, 74729.31it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
  assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
  picks = []
  for _ in range(num_examples):
    pick = random.randint(0, len(dataset)-1)
    while pick in picks:
      pick = random.randint(0, len(dataset)-1)
    picks.append(pick)

  df = pd.DataFrame(dataset[picks])
  display(HTML(df.to_html()))


In [7]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,そこに彼は彼の弟、ロバートのキャリアを進めるために彼ができたすべてをしました。
1,ブレチルの次の首長は甥のハルダンだった。
2,私は我々の概念的知識は手から得られたのであると思う。
3,動作緩徐は実行機能障害、作業記憶障害、注意障害よりも先に表れている。
4,他のチャンネルも似たようなドラマかワイドショーが流れていた
5,メニューを大幅に簡素化した
6,閉会式で泣き出したコンパニオンも、夜のディスコパーティーでは、激しく踊りまくった。
7,年老いた茶色と赤のコートを着たカップルが、忙しい通りを歩いています。
8,かつて、パッテンは、ロンドンのぬかるんだ道のことで交渉をする歩行者の助けになりました。
9,「整然とした原理」は「秩序定理」と同義であると解釈されることがある。


In [8]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [9]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/10990 [00:00<?, ? examples/s]

Map:   0%|          | 0/4604 [00:00<?, ? examples/s]

In [10]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,日本人はおはしを使って、ごはんを食べます。
1,冷蔵庫の中に野菜や卵があります。
2,その他の舞台には「exit muttering」「crash」「minuet for brass band」などがあった。
3,彼は博士号を取得し、法律家としての短いキャリアを開始した。
4,映画の慎重なデザインはこれを軽減することができます。
5,そうざい売り場が充実している
6,彼はプリスティナとカメニカに住んでいます。
7,きょうはもう家へ帰ります。
8,このような分析は、法医学工学において大いに有用。
9,このスマホゲー、まだやってたんだ


In [11]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [12]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/10990 [00:00<?, ? examples/s]

Map:   0%|          | 0/4604 [00:00<?, ? examples/s]

In [13]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [14]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'鍛': 0,
 '双': 1,
 '臆': 2,
 '聞': 3,
 '苗': 4,
 '泣': 5,
 '荊': 6,
 '腫': 7,
 '唸': 8,
 '鷲': 9,
 '銀': 10,
 '滑': 11,
 '級': 12,
 '府': 13,
 '完': 14,
 '錬': 15,
 '鯨': 16,
 'r': 17,
 '飾': 18,
 '歌': 19,
 '怒': 20,
 '世': 21,
 '組': 22,
 '勃': 23,
 '待': 24,
 '網': 25,
 '逐': 26,
 '祭': 27,
 '析': 28,
 '系': 29,
 '息': 30,
 '停': 31,
 'ぇ': 32,
 '古': 33,
 '豆': 34,
 '－': 35,
 '滞': 36,
 '秩': 37,
 '紫': 38,
 '因': 39,
 '潟': 40,
 '刊': 41,
 '用': 42,
 '棄': 43,
 '了': 44,
 '論': 45,
 '境': 46,
 '雰': 47,
 '泉': 48,
 '苔': 49,
 '周': 50,
 '演': 51,
 '債': 52,
 'ざ': 53,
 '簡': 54,
 '吹': 55,
 '句': 56,
 '五': 57,
 '机': 58,
 '影': 59,
 '粘': 60,
 '兼': 61,
 '持': 62,
 '捕': 63,
 'ｆ': 64,
 '足': 65,
 '肉': 66,
 '線': 67,
 '．': 68,
 '醸': 69,
 '富': 70,
 '驚': 71,
 '温': 72,
 '貴': 73,
 '麦': 74,
 '画': 75,
 '飽': 76,
 '箔': 77,
 '醍': 78,
 '礫': 79,
 '魂': 80,
 '朗': 81,
 '燕': 82,
 '億': 83,
 '末': 84,
 '柳': 85,
 'ず': 86,
 '履': 87,
 '評': 88,
 '蛙': 89,
 '模': 90,
 '六': 91,
 '壁': 92,
 '谷': 93,
 '京': 94,
 '集': 95,
 '暫': 96,
 '銘': 97,
 '獣': 98,
 '盟': 99,
 '茅': 100,

In [15]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [16]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

2438

In [17]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [18]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

## XLSR-Wav2Vec2 Feature Extractor

In [19]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [20]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [21]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
processor.save_pretrained('/content/drive/MyDrive/wav2vec2-large-xlsr-japanese-demo')

[]

## Preprocess Data

In [23]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/0fb75edb8bd454df8bb23ae8bee20532bae3434c4b63d0b4e126d443f35c56e7/ja_train_0/common_voice_ja_25861545.mp3'

XLSR-Wav2Vec2モデルは、入力として16 kHzのサンプリングレートを期待しています。つまり、オーディオファイルをロードする際に、そのファイルが16 kHzのサンプリングレートであることを保証する必要があります。もしオーディオファイルのサンプリングレートが16 kHzでない場合は、リサンプリングが必要になります。

In [24]:
common_voice_train[0]["audio"] # サンプリングレートが 48kHz(48000)

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/0fb75edb8bd454df8bb23ae8bee20532bae3434c4b63d0b4e126d443f35c56e7/ja_train_0/common_voice_ja_25861545.mp3',
 'array': array([ 0.00000000e+00, -9.94759830e-14,  3.19744231e-14, ...,
        -1.35150884e-07, -2.39865514e-08, -3.71753970e-08]),
 'sampling_rate': 48000}

In [25]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: 反対の概念は過剰です。 
Input array shape: (207360,)
Sampling rate: 48000


In [26]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [27]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/0fb75edb8bd454df8bb23ae8bee20532bae3434c4b63d0b4e126d443f35c56e7/ja_train_0/common_voice_ja_25861545.mp3'

In [28]:
common_voice_train[0]["audio"] # サンプリングレートの確認 16kHz

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/0fb75edb8bd454df8bb23ae8bee20532bae3434c4b63d0b4e126d443f35c56e7/ja_train_0/common_voice_ja_25861545.mp3',
 'array': array([ 0.00000000e+00,  3.63797881e-12,  2.72848411e-12, ...,
        -7.86050691e-07, -8.68930897e-07, -2.41277917e-07]),
 'sampling_rate': 16000}

In [29]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [30]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [31]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)
# prepare_dataset関数をデータセットに適応

Map (num_proc=4):   0%|          | 0/10990 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/4604 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

## Training

In [32]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [33]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [34]:
wer_metric = load_metric("wer")

<ipython-input-34-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [35]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [36]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1920: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [38]:
model.freeze_feature_extractor()

In [39]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-0zk1hpn8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-0zk1hpn8
  Resolved https://github.com/huggingface/accelerate to commit d9a1b8f97504efad8055693fd2311aa5199791ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.29.0.dev0-py3-none-any.whl size=296450 sha256=a113d1235f0d955729b2ec858b93250343affba860b6a32f0cff353361023484
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_ea_6y2/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [40]:
import accelerate
print(accelerate.__version__)

0.29.0.dev0


In [43]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-japanese-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`